### Subwords Tokenizing
- google sentencepiece
- huggingface sentencepiece
- normally 25,000 ~ 35,000 words

### 종류

#### 1. BPE (Byte-Pair Encoding)
- 공백 기준으로 pre-tokenize를 거친 후 가장 함께 많이 등장한 character pair를 사전에 추가

#### 2. Wordpiece
- bert에서 사용
- 사용자가 지정한 횟수 만큼 subwords를 졍합하는 방식. bpe와 다른 점은 bpe는 같이 많이 등장한 쌍을 병합하지만 wordpiece는 병합되었을 때 corpus의 likelihood를 가장 높이는 쌍을 병합

#### 3. Sentencepiece
- wordpiece, sentencepiece는 BPE의 작은 변형들
    ##### Unigram
    - bpe, wordpiece와 같이 기본 character에서 subwords를 점진적으로 병합하는 것이 아니라, 모든 pre-tokenized token과 subwords에서 시작해 점차 사전을 줄여나가는 방식으로 진행
    - SentencePiece에서 활용되는 알고리즘
    - 대부분 한국어는 wordpiece보다 sentencepiece를 이용한다

### make corpus

In [1]:
from tqdm import tqdm
from dbConn.mongo_conn import config

conn = config()
col = conn["travel_ai"].blog_contents

In [2]:
contents = col.find({"num_docs": {"$gt": 1}}, {"cleaned_content": 1})
input_f = "./data/for_tokenizer_corpus.txt"
f = open(input_f, "w")
for cont in contents:
    docs = [c for c in cont['cleaned_content']]
    f.write('\n'.join(docs))
del contents

f.close()
conn.close()

162013it [00:03, 44571.96it/s]


108118909

# google sentencepiece

In [10]:
%%time
import os
import sentencepiece as spm

vocab_size = 35000
sp_model_root = './data/tokenizer/'
if not os.path.isdir(sp_model_root):
    os.mkdir(sp_model_root)
sp_model_name = 'blog_sentpiece_%d' % (vocab_size)
sp_model_path = os.path.join(sp_model_root, sp_model_name)
model_type = 'unigram' # or "bpe"
character_coverage  = 0.9995
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS],[UNK0],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5],[UNK6],[UNK7],[UNK8],[UNK9],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],[unused9],[unused10],[unused11],[unused12],[unused13],[unused14],[unused15],[unused16],[unused17],[unused18],[unused19],[unused20],[unused21],[unused22],[unused23],[unused24],[unused25],[unused26],[unused27],[unused28],[unused29],[unused30],[unused31],[unused32],[unused33],[unused34],[unused35],[unused36],[unused37],[unused38],[unused39],[unused40],[unused41],[unused42],[unused43],[unused44],[unused45],[unused46],[unused47],[unused48],[unused49],[unused50],[unused51],[unused52],[unused53],[unused54],[unused55],[unused56],[unused57],[unused58],[unused59],[unused60],[unused61],[unused62],[unused63],[unused64],[unused65],[unused66],[unused67],[unused68],[unused69],[unused70],[unused71],[unused72],[unused73],[unused74],[unused75],[unused76],[unused77],[unused78],[unused79],[unused80],[unused81],[unused82],[unused83],[unused84],[unused85],[unused86],[unused87],[unused88],[unused89],[unused90],[unused91],[unused92],[unused93],[unused94],[unused95],[unused96],[unused97],[unused98],[unused99]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s --character_coverage=%s'
cmd = input_argument%(input_f, sp_model_path, vocab_size,user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd)
print('train done')

train done
CPU times: user 55min 14s, sys: 12.1 s, total: 55min 26s
Wall time: 17min 8s


In [11]:
## check
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(sp_model_path))

test_txt = "사장님 추천메뉴 복숭아요거트복숭아 쨈도 완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요"
tokens = sp.encode_as_pieces(test_txt)
ids = sp.encode_as_ids(test_txt)

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[1286, 23664, 9404, 13793, 23982, 121, 17852, 122, 404, 18397, 150, 1138, 18044, 3950, 122, 9054, 188, 5800, 456]
['▁사장님', '▁추천메뉴', '▁복숭아', '요거트', '복숭아', '▁', '쨈', '도', '▁완전', '▁상콤', '하고', '▁맛있고', '▁겉에', '▁과자', '도', '▁바삭바삭', '해서', '▁식감도', '▁좋았어요']
사장님 추천메뉴 복숭아요거트복숭아 쨈도 완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요
사장님 추천메뉴 복숭아요거트복숭아 쨈도 완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요


# Huggingface tokenizer_bertwordpiece
- Rust로 구현되어 있어 빠르다.

In [16]:
# !pip install tokenizers

     |████████████████████████████████| 6.8 MB 16.5 MB/s            


In [3]:
%%time
input_f = "./data/for_tokenizer_corpus.txt"
with open(input_f, 'r') as f:
    data = f.read().splitlines()
print(len(data))
print(data[:3])

3586068
['까막바위', '강원도 동해시 묵호동동해시 까막바위 일출 해맞이 해돋이 겨울바다 한파', '까막바위에서~까막바위']
CPU times: user 1.48 s, sys: 512 ms, total: 1.99 s
Wall time: 1.99 s


wordpiece 사용시 mecab으로 토큰 후 wordpiece 훈련하는 것이 더 적합할 수 있따.

In [4]:
%%time
from konlpy.tag import Mecab
m = Mecab()

for_generation = False # or normal

if for_generation:
    # 1: '어릴때' -> '어릴, ##때' for generation model
    total_morph=[]
    for sentence in data:
        morph_sentence= []
        count = 0
        for token_mecab in m.morphs(sentence):
            token_mecab_save = token_mecab
            if count > 0:
                token_mecab_save = "##" + token_mecab_save
                morph_sentence.append(token_mecab_save)
            else:
                morph_sentence.append(token_mecab_save)
                count += 1
        total_morph.append(morph_sentence)

else:
    # 2: '어릴때' -> '어릴, 때'   for normal case
    total_morph=[]
    for sentence in data:
        morph_sentence= m.morphs(sentence)
        total_morph.append(morph_sentence)
                        
print(total_morph[:3])
print(len(total_morph))

with open('./data/for_tokenizer_corpus_mecab.txt', 'w', encoding='utf-8') as f:
    for line in total_morph:
        f.write(' '.join(line)+'\n')

[['까막', '바위'], ['강원도', '동해시', '묵호동', '동해시', '까막', '바위', '일출', '해맞이', '해돋이', '겨울', '바다', '한파'], ['까막', '바위', '에서', '~', '까막', '바위']]
3586068
CPU times: user 4min 42s, sys: 2.52 s, total: 4min 44s
Wall time: 4min 44s


- Dummy token -> [unused], [UNK] 설정
    - [BOS] 문장 시작 [EOS] 문장 끝
    - 특히 도메인 특화 task 수행시 반드시 도메인 토큰을 따로 선언

- CharBPETokenizer: The original BPE
- ByteLevelBPETokenizer: The byte level version of the BPE
- SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
- BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece
<br>
- 한국어는 strip_accents = False로 해줘야 한다
    - 만약 True일 시 나는 -> 'ㄴ','ㅏ','ㄴ','ㅡ','ㄴ' 로 쪼개져서 처리된다
    - 학습시 False했으므로 load할 때도 False를 꼭 확인
- BertWordPieceTokenizer로 학습시킬 땐 lower_case=False 시 strip_accent=False로 지정.<br> huggingface transformer에서 tokenzizer를 load할 때도 strip_accent=False를 꼭 지정해야한다

In [5]:
%%time
import os
from tokenizers import BertWordPieceTokenizer, SentencePieceBPETokenizer, CharBPETokenizer, ByteLevelBPETokenizer

how_to_tokenize = BertWordPieceTokenizer

if str(how_to_tokenize) == str(BertWordPieceTokenizer):
    print('BertWordPieceTokenizer')
    tokenizer = BertWordPieceTokenizer(strip_accents=False,  # Must be False if cased model
                                       lowercase=False)
elif str(how_to_tokenize) == str(SentencePieceBPETokenizer):
    print('SentencePieceBPETokenizer')
    tokenizer = SentencePieceBPETokenizer()

elif str(how_to_tokenize) == str(CharBPETokenizer):
    print('CharBPETokenizer')
    tokenizer = CharBPETokenizer()
    
elif str(how_to_tokenize) == str(ByteLevelBPETokenizer):
    print('ByteLevelBPETokenizer')
    tokenizer = ByteLevelBPETokenizer()
       
else:
    assert('select right tokenizer')

BertWordPieceTokenizer
CPU times: user 5.59 ms, sys: 91 µs, total: 5.68 ms
Wall time: 28.2 ms


- huggingface tokinizer params
    - min_frequency : merge를 수행할 최소 빈도수, 5로 설정 시 5회 이상 등장한 pair만 수행한다
    - vocab_size: 만들고자 하는 vocab의 size, 보통 '32000' 정도가 좋다고 알려져 있다
    - show_progress : 학습 진행과정 show
    - special_tokens : Tokenizer에 추가하고 싶은 special token 지정
    - limit_alphabet : merge 수행 전 initial tokens이 유지되는 숫자 제한. ByteLevelBPETokenizer 학습시엔 주석처리 필요
    - initial_alphabet : 꼭 포함됐으면 하는 initial alphabet, 이곳에 설정한 token은 학습되지 않고 그대로 포함되도록 설정된다.

In [6]:
corpus_file   = ["./data/for_tokenizer_corpus_mecab.txt"]  # data path
vocab_size    = 30000 # 35000 #######
limit_alphabet= 5000 # 6000 #######
output_path   = 'hugging_%d'%(vocab_size)
min_frequency = 2 # 5

# train
tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet,
               show_progress=True)
print('train complete')

test_txt = "사장님 추천메뉴 복숭아요거트복숭아 쨈도 완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요"
sentence = test_txt
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

sentence = 'I want to go my hometown'
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

# save tokenizer
hf_model_path='hf_tokenizer'
if not os.path.isdir(hf_model_path):
    os.mkdir(hf_model_path)
tokenizer.save_model(hf_model_path)  # vocab.txt 파일 한개가 만들어진다

train complete
사장님 추천메뉴 복숭아요거트복숭아 쨈도 완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요
=>idx   : [6604, 4385, 6530, 12704, 10138, 4262, 4314, 4189, 4201, 10020, 3025, 4237, 6623, 2001, 4725, 17080, 6474, 4213, 142, 4444, 8155, 4237, 8556, 6991, 6885, 4237, 2871, 5651, 6682]
=>tokens: ['사장', '##님', '추천', '##메뉴', '복숭아', '##요', '##거', '##트', '##복', '##숭아', '쨈', '##도', '완전', '상', '##콤', '##하고', '맛있', '##고', '겉', '##에', '과자', '##도', '바삭바삭', '##해서', '식감', '##도', '좋', '##았', '##어요']
=>offset: [(0, 2), (2, 3), (4, 6), (6, 8), (9, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 18), (19, 20), (20, 21), (22, 24), (25, 26), (26, 27), (27, 29), (30, 32), (32, 33), (34, 35), (35, 36), (37, 39), (39, 40), (41, 45), (45, 47), (48, 50), (50, 51), (52, 53), (53, 54), (54, 56)]
=>decode: 사장님 추천메뉴 복숭아요거트복숭아 쨈도 완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요

I want to go my hometown
=>idx   : [31, 71, 22082, 13757, 12466, 8192, 8854, 4200, 20770]
=>tokens: ['I', 'w', '##ant', 'to', 'go', 'my', 'home', '##t', '##own']
=>offset: [(

['hf_tokenizer/vocab.txt']

In [ ]:
# !pip install transformers

In [7]:
# transformer usage check
from transformers import BertTokenizerFast

loaded_tokenizer = BertTokenizerFast.from_pretrained(hf_model_path, strip_accents=False, lowercase=False)
print('voca size : %d' % loaded_tokenizer.vocab_size)
tokenized_input_for_torch = loaded_tokenizer('과자가 완전 바삭', return_tensors='pt')

print('Token (str) : {}'.format([loaded_tokenizer.convert_ids_to_tokens(s) for s in tokenized_input_for_torch['input_ids'].tolist()[0]]))
print('Token (int) : {}'.format([tokenized_input_for_torch['input_ids'].tolist()[0]]))
print('Token (attn_mask) : {}'.format([tokenized_input_for_torch['attention_mask'].tolist()[0]]))

file hf_tokenizer/config.json not found
file hf_tokenizer/config.json not found


voca size : 30000
Token (str) : ['[CLS]', '과자', '##가', '완전', '바삭', '[SEP]']
Token (int) : [[2, 8155, 4391, 6623, 7016, 3]]
Token (attn_mask) : [[1, 1, 1, 1, 1, 1]]


In [12]:
import random
random.sample(list(loaded_tokenizer.get_vocab().items()), 10) # 랜덤으로 10개 확인

[('Co', 27554),
 ('할리', 21975),
 ('수목', 9956),
 ('남이섬', 17773),
 ('버티', 16050),
 ('귑', 271),
 ('대한민국', 9415),
 ('쉑쉑', 12522),
 ('녹', 639),
 ('방면', 12326)]

In [9]:
user_defined_symbols = ['[PAD]', '[UNK]', '[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]', '[CLS]', '[SEP]', '[MASK]', '[BOS]','[EOS]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

In [10]:
loaded_tokenizer.all_special_tokens # 사용자 정의 심볼 train시 등록해도 등록X. load해서 사용시 따로 추가해줘야함 

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [14]:
# user define symbol 추가하는 방법
special_token_dic = {'additional_special_tokens': user_defined_symbols}
loaded_tokenizer.add_special_tokens(special_token_dic)

loaded_tokenizer.all_special_tokens[-10:]

['[unused190]',
 '[unused191]',
 '[unused192]',
 '[unused193]',
 '[unused194]',
 '[unused195]',
 '[unused196]',
 '[unused197]',
 '[unused198]',
 '[unused199]']

In [15]:
# special token 추가 후 저장
loaded_tokenizer.save_pretrained(hf_model_path+'_special')

('hf_tokenizer_special/tokenizer_config.json',
 'hf_tokenizer_special/special_tokens_map.json',
 'hf_tokenizer_special/vocab.txt',
 'hf_tokenizer_special/added_tokens.json',
 'hf_tokenizer_special/tokenizer.json')

In [17]:
# check special tokens
from transformers import BertTokenizerFast
tokenizer_check = BertTokenizerFast.from_pretrained('hf_tokenizer'+'_special')

print('check special tokens : %s'%tokenizer_check.all_special_tokens[:20])

print('vocab size : %d' % tokenizer_check.vocab_size)
tokenized_input_for_pytorch = tokenizer_check("완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요", return_tensors="pt")

print("Tokens (str)      : {}".format([tokenizer_check.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))

check special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[BOS]', '[EOS]', '[unused0]', '[unused1]', '[unused2]']
vocab size : 30000
Tokens (str)      : ['[CLS]', '완전', '상', '##콤', '##하고', '맛있', '##고', '겉', '##에', '과자', '##도', '바삭바삭', '##해서', '식감', '##도', '좋', '##았', '##어요', '[SEP]']
Tokens (int)      : [2, 6623, 2001, 4725, 17080, 6474, 4213, 142, 4444, 8155, 4237, 8556, 6991, 6885, 4237, 2871, 5651, 6682, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [18]:
# torch에 input으로 잘 들어가는지 확인
from transformers import BertModel

model = BertModel.from_pretrained('bert-base-cased') # test용 모델
input_sent = tokenizer_check("완전 상콤하고 맛있고 겉에 과자도 바삭바삭해서 식감도 좋았어요.", return_tensors="pt") # pt: pytorch, tf: tensorflow
ouput_sent = model(**input_sent)
print('final layer output shape : %s' % (ouput_sent['last_hidden_state'].shape,))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


final layer output shape : torch.Size([1, 20, 768])
